In [1]:
import os
import numpy as np
import pandas as pd
from backtrader_bokeh import bt

from src.defs import BROKER
from src.utils import logger, load_oanda_parquet

os.environ["BOKEH_ALLOW_WS_ORIGIN"] = '0lj9hh483va927cadklatiu5affjj5cn3b349he4cl71d36qc4p6'

logger.setLevel(10) # debug



***************: The token is incorrect, you will not be able to use advanced features, such as: 'logger', 'configer', 'keyboard control' and more... 

You can purchase Bakctrader_Bokeh on website: https://aui.photos/backtrader-bokeh/purchase/ 



Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
from src.strategy import MyStrategy

INSTRUMENT = "EUR_USD"
DATA_FILE = 'oanda_EUR_USD_M1_2022-12-19_2022-12-31.parquet.gz'
DATA_NAME = 'EUR_USD_H1'

TIMEFRAME = bt.TimeFrame.Minutes
COMPRESSION = 60

# initialize
cerebro = bt.Cerebro()
cerebro.broker.setcash(1000)

# set commission and margin
cerebro.broker.setcommission(
    commission=BROKER.COMMISSION[INSTRUMENT],
    commtype=bt.CommInfoBase.COMM_PERC,
    percabs=True, # understand commission as 0.XX
    leverage=50,
    )

# add data
df = load_oanda_parquet(DATA_FILE)

data = bt.feeds.PandasData(
    dataname=df,
    name=DATA_NAME,
)

cerebro.replaydata(data, timeframe=TIMEFRAME, compression=COMPRESSION)

# add strategy
cerebro.addstrategy(MyStrategy)

# add analyzers
cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='Trades')

# add observer
cerebro.addobserver(bt.observers.BuySell, barplot=True, bardist=0)

# run
results = cerebro.run()
result = results[0]

# save trade log
df_trade_log = pd.DataFrame(result.trade_log)
df_trade_log.to_csv('__temp_trade_log.csv', index=False)

[utils.py:38 - load_oanda_parquet()] Loaded data has 12741 rows, from 2022-12-19 00:00:00+00:00 to 2022-12-30 21:58:00+00:00.


STRATEGY START: value = 1000, cash = 1000.
[2022-12-22 13:11:00] SELL MKT ORDER TRIGGERED.
[2022-12-22 13:12:00] SELL MKT ORDER ACCEPTED: price = 1.063325, size = 3603.0, margin = None.
[2022-12-22 13:12:00] BUY STOP ORDER ACCEPTED: price = 1.0661, size = 3603.0, margin = None.
[2022-12-22 13:12:00] BUY LMT ORDER ACCEPTED: price = 1.06055, size = 3603.0, margin = None.
[2022-12-22 13:12:00] SELL MKT ORDER EXECUTED: order_price = 1.06333, execute_price = 1.06334, slippage = -0.1 %%, size = 3603.0, value = 3831.21402, margin = 1.0, commission = 0.28734.
[2022-12-22 13:36:00] BUY LMT ORDER EXECUTED: order_price = 1.06055, execute_price = 1.06055, slippage = 0.0 %%, size = 3603.0, value = 3821.16165, margin = 1.0, commission = 0.28659.
[2022-12-22 13:36:00] BUY STOP ORDER CANCELLED.
[2022-12-22 13:36:00] TRADE 25 CLOSED: gross_pnl = 10.05237, net_pnl = 9.47844.
[2022-12-23 02:34:00] BUY MKT ORDER TRIGGERED.
[2022-12-23 02:35:00] BUY MKT ORDER ACCEPTED: price = 1.0612, size = 2713.0, margin

In [ ]:
bullish_color = '#31a354' # "green" 
bearish_color = '#e6550d' # "red"
bokeh_plotter = bt.Bokeh(barup=bullish_color, barup_outline=bullish_color, barup_wick=bullish_color, 
                         bardown=bearish_color, bardown_outline=bearish_color, bardown_wick=bearish_color)
cerebro.plot(plotter=bokeh_plotter, iplot=False)

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.2)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


[{0: <backtrader_bokeh.figure.FigurePage at 0x17381fba190>}]

[web.py:2271 - log_request()] 200 GET /autoload.js?bokeh-autoload-element=3452&bokeh-absolute-url=http://localhost:62046&resources=none (::1) 377.16ms
[web.py:2271 - log_request()] 101 GET /ws?id=5ad1b171-8162-4fd7-a832-4ad250577627&origin=bce4bbf1-101d-44db-883b-e3db4f31e0a2&swVersion=4&extensionId=&platform=electron&vscode-resource-base-authority=vscode-resource.vscode-cdn.net&parentOrigin=vscode-file%3A%2F%2Fvscode-app&purpose=notebookRenderer (::1) 16.69ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created


In [ ]:
print('Total Trades:', result.analyzers.Trades.get_analysis()['total'])
print('Total Returns:', result.analyzers.Returns.get_analysis()['rtot'])
print('Max Drawdown:', result.analyzers.DrawDown.get_analysis()['max']['drawdown'])
print('Sharpe Ratio:', result.analyzers.SharpeRatio.get_analysis()['sharperatio'])